In [2]:
from pyiceberg import __version__

__version__

'0.9.1'

## 

In [5]:
import pyarrow.parquet as pq
import pyarrow.dataset as ds
ds.JsonFileFormat()
tbl_taxis = ds.dataset('/home/jovyan/notebooks/testdata/consuption/',format='json')
tbl_taxis

ArrowInvalid: Error creating dataset. Could not read schema from '/home/jovyan/notebooks/testdata/consuption/ConsumptionIndustry0.json'. Is this a 'json' file?: Could not open JSON input source '/home/jovyan/notebooks/testdata/consuption/ConsumptionIndustry0.json': Invalid: straddling object straddles two block boundaries (try to increase block size?)

## Creating the table

Next, create the namespace, and the `taxis` table from the schema that's derived from the Arrow schema:

In [ ]:
from pyiceberg.catalog import load_catalog
from pyiceberg.exceptions import NamespaceAlreadyExistsError

cat = load_catalog('default')

try:
    cat.create_namespace('default')
except NamespaceAlreadyExistsError:
    pass

In [ ]:
from pyiceberg.exceptions import NoSuchTableError

try:
    cat.drop_table('default.taxis')
except NoSuchTableError:
    pass

tbl = cat.create_table(
    'default.taxis',
    schema=tbl_taxis.schema
)

tbl

## Write the actual data into the table

This will create a new snapshot on the table:

In [ ]:
tbl.overwrite(tbl_taxis)

tbl

## Append more data

Let's append another month of data to the table:

In [ ]:
tbl.append(pq.read_table('/home/iceberg/data/yellow_tripdata_2021-05.parquet'))
tbl

## Load data into a PyArrow Dataframe

We'll fetch the table using the REST catalog that comes with the setup.

In [ ]:
tbl = cat.load_table('default.taxis')

sc = tbl.scan(row_filter="tpep_pickup_datetime >= '2021-05-01T00:00:00.000000'")

In [ ]:
df = sc.to_arrow().to_pandas()

In [ ]:
len(df)

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.hist(column='fare_amount')

In [ ]:
import numpy as np
from scipy import stats

stats.zscore(df['fare_amount'])

# Remove everything larger than 3 stddev
df = df[(np.abs(stats.zscore(df['fare_amount'])) < 3)]
# Remove everything below zero
df = df[df['fare_amount'] > 0]

In [ ]:
df.hist(column='fare_amount')

# DuckDB

Use DuckDB to Query the PyArrow Dataframe directly.

In [ ]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [ ]:
%sql SELECT * FROM df LIMIT 20

In [ ]:
%%sql --save tip_amount --no-execute

SELECT tip_amount
FROM df

In [ ]:
%sqlplot histogram --table df --column tip_amount --bins 22 --with tip_amount

In [ ]:
%%sql --save tip_amount_filtered --no-execute

WITH tip_amount_stddev AS (
    SELECT STDDEV_POP(tip_amount) AS tip_amount_stddev
    FROM df
)

SELECT tip_amount
FROM df, tip_amount_stddev
WHERE tip_amount > 0
  AND tip_amount < tip_amount_stddev * 3

In [ ]:
%sqlplot histogram --table tip_amount_filtered --column tip_amount --bins 50 --with tip_amount_filtered
